In [6]:
#Load library
import pandas as pd 
import numpy as np
import requests
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import tweepy
%matplotlib inline

In [16]:
#Read @WeRateDogs twitter acrhive 
df_archive = pd.read_csv("twitter-archive-enhanced.csv")

In [46]:
df_archive.shape

(2356, 17)

In [4]:
#Connect Twitter APIs to query more data related to archived tweets
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

In [32]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [75]:
tweet_info = []
for n in range(100,df_archive.shape[0]+100,100):
    tweet_id = df_archive.tweet_id[:n].tolist()
    lookups = api.statuses_lookup(id_=tweet_id)
    for info in lookups:
        tweet_info = tweet_info.apend({"tweet_id": info.id})
#     lookups = lookups.append(api.statuses_lookup(id_=tweet_id))
    print(tweet_info)
    break
        
# for t_id in tweet_id:
#     lookups = api.statuses_lookup(id_=t_id)
# df_archive.tweet_id[0:10]

AttributeError: 'list' object has no attribute 'apend'

TypeError: 'NoneType' object is not subscriptable